# **The recommendation engines present in this model are:**

**1. Simple Recommender System :**
Sorts movies based on ratings, genre and popularity .Displays the top movies of the list.

**2. Movie Description based Recommender System :**
Computes similarity between movies description and is used to suggest movies that are most similar to a particular movie that a user liked. Used TfidfVectorizer to transform text to feature vectors that can be used as input to estimator.

**3. Credits and Keywords based Recommender :**
Recommendations are given based on keywords.

**4. Collaborative filtering based Recommender System :**
Records from various users provide recommendations based on user similarities.

**5. Hybrid filtering based Recommender System :**
This filtering takes advantage of content-based and collaborative filtering.
It provides different recommendations for different users even though the movie title is the same.

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/the-movies-dataset/links.csv
/kaggle/input/the-movies-dataset/ratings_small.csv
/kaggle/input/the-movies-dataset/links_small.csv
/kaggle/input/the-movies-dataset/credits.csv
/kaggle/input/the-movies-dataset/ratings.csv
/kaggle/input/the-movies-dataset/movies_metadata.csv
/kaggle/input/the-movies-dataset/keywords.csv


In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
from sklearn.feature_extraction.text import CountVectorizer
from surprise import Reader, Dataset, SVD
from sklearn.metrics.pairwise import cosine_similarity
from surprise.model_selection import cross_validate
from ast import literal_eval

In [3]:
df=pd.read_csv('/kaggle/input/the-movies-dataset/movies_metadata.csv')
df.head()

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


# **Simple Recommender System**

Sort movies based on ratings,popularity and genre.Display the top movies of the list.

In [4]:
C= df['vote_average'].mean()
print('Average Vote: ', C)
m= df['vote_count'].quantile(0.95)
print('Vote Count above 95% : ', m)

Average Vote:  5.618207215133889
Vote Count above 95% :  434.0


In [5]:
qualified = df.copy().loc[df['vote_count'] >= m]
qualified.shape

(2274, 24)

In [6]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)

qualified['score'] = qualified.apply(weighted_rating, axis=1)
qualified = qualified.sort_values('score', ascending=False).head(100)
qualified[['title', 'vote_count', 'vote_average', 'score']].head(10)

,title,vote_count,vote_average,score
314,The Shawshank Redemption,8358.0,8.5,8.357746
834,The Godfather,6024.0,8.5,8.306334
12481,The Dark Knight,12269.0,8.3,8.208376
2843,Fight Club,9678.0,8.3,8.184899
292,Pulp Fiction,8670.0,8.3,8.172155
351,Forrest Gump,8147.0,8.2,8.069421
522,Schindler's List,4436.0,8.3,8.061007
23673,Whiplash,4376.0,8.3,8.058025
5481,Spirited Away,3968.0,8.3,8.035598
1154,The Empire Strikes Back,5998.0,8.2,8.025793


In [7]:
df['genres']

0        [{'id': 16, 'name': 'Animation'}, {'id': 35, '...
1        [{'id': 12, 'name': 'Adventure'}, {'id': 14, '...
2        [{'id': 10749, 'name': 'Romance'}, {'id': 35, ...
3        [{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...
4                           [{'id': 35, 'name': 'Comedy'}]
                               ...                        
45461    [{'id': 18, 'name': 'Drama'}, {'id': 10751, 'n...
45462                        [{'id': 18, 'name': 'Drama'}]
45463    [{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...
45464                                                   []
45465                                                   []
Name: genres, Length: 45466, dtype: object

In [8]:
def genre_based(genre, percentile=0.85):
    vote_counts = df[df['vote_count'].notnull()]['vote_count'].astype('int')
    vote_averages = df[df['vote_average'].notnull()]['vote_average'].astype('int')
    C = vote_averages.mean()
    m = vote_counts.quantile(percentile)
    
    qualified = df[(df['vote_count'] >= m) ] [['title', 'vote_count', 'vote_average', 'popularity']]
    qualified['vote_count'] = qualified['vote_count'].astype('int')
    qualified['vote_average'] = qualified['vote_average'].astype('int')
    qualified['score'] = qualified.apply(lambda x: (x['vote_count']/(x['vote_count']+m) * x['vote_average'])  + (m/(m+x['vote_count']) * C), axis=1)
    qualified = qualified.sort_values('score', ascending=False).head(250)
    
    return qualified

In [9]:
genre_based('Romance').head(15)

,title,vote_count,vote_average,popularity,score
10309,Dilwale Dulhania Le Jayenge,661,9,34.457,8.585574
15480,Inception,14075,8,29.1081,7.984042
12481,The Dark Knight,12269,8,123.167,7.981708
22879,Interstellar,11187,8,32.2135,7.979952
2843,Fight Club,9678,8,63.8696,7.976853
4863,The Lord of the Rings: The Fellowship of the Ring,8892,8,32.0707,7.974825
292,Pulp Fiction,8670,8,140.95,7.974187
314,The Shawshank Redemption,8358,8,51.6454,7.973232
7000,The Lord of the Rings: The Return of the King,8226,8,29.3244,7.972807
351,Forrest Gump,8147,8,48.3072,7.972546


# **Content Based Recommendation System**

This system computes similarity between movies such that it is used to suggest movies that are most similar to a particular movie that a user liked.
A smaller dataset is used due to lack of computational power.

In [10]:
df_small =pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv')
df_small

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9120,162672,3859980,402672.0
9121,163056,4262980,315011.0
9122,163949,2531318,391698.0
9123,164977,27660,137608.0


In [11]:
df_small = df_small[df_small['tmdbId'].notnull()]['tmdbId'].astype('int')
df = df.drop([19730, 29503, 35587])
df['id'] = df['id'].astype('int')
sdf = df[df['id'].isin(df_small)]
sdf.shape

(9099, 24)

In [12]:
sdf

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40224,False,NaN,15000000,"[{'id': 28, 'name': 'Action'}, {'id': 12, 'nam...",NaN,315011,tt4262980,ja,シン・ゴジラ,From the mind behind Evangelion comes a hit la...,...,2016-07-29,77000000.0,120.0,"[{'iso_639_1': 'it', 'name': 'Italiano'}, {'is...",Released,A god incarnate. A city doomed.,Shin Godzilla,False,6.6,152.0
40503,False,NaN,0,"[{'id': 99, 'name': 'Documentary'}, {'id': 104...",http://www.thebeatlesliveproject.com/,391698,tt2531318,en,The Beatles: Eight Days a Week - The Touring Y...,"The band stormed Europe in 1963, and, in 1964,...",...,2016-09-15,0.0,99.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,The band you know. The story you don't.,The Beatles: Eight Days a Week - The Touring Y...,False,7.6,92.0
44821,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",16000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://movies.warnerbros.com/pk3/,10991,tt0235679,ja,Pokémon 3: The Movie,When Molly Hale's sadness of her father's disa...,...,2000-07-08,68411275.0,93.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Pokémon: Spell of the Unknown,Pokémon: Spell of the Unknown,False,6.0,144.0
44826,False,"{'id': 34055, 'name': 'Pokémon Collection', 'p...",0,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",http://www.pokemon.com/us/movies/movie-pokemon...,12600,tt0287635,ja,劇場版ポケットモンスター セレビィ 時を越えた遭遇（であい）,"All your favorite Pokémon characters are back,...",...,2001-07-06,28023563.0,75.0,"[{'iso_639_1': 'ja', 'name': '日本語'}]",Released,NaN,Pokémon 4Ever: Celebi - Voice of the Forest,False,5.7,82.0


In [13]:
sdf['description'] = sdf['overview'] + sdf['tagline']
sdf['description'] = sdf['description'].fillna('')
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix = tf.fit_transform(sdf['description'])
tfidf_matrix.shape

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(9099, 217961)

In [14]:
cos_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
sdf = sdf.reset_index()
indices = pd.Series(sdf.index, index=sdf['title']).drop_duplicates()

def get_recommendations(title, cos_sim=cos_sim):
    idx = indices[title]
    sim_scores = list(enumerate(cos_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:11]
    movie_indices = [i[0] for i in sim_scores]
    return sdf['title'].iloc[movie_indices]

In [15]:
get_recommendations('The Baby-Sitters Club')

4374             Best Friends
431                    Lassie
2658      The Next Best Thing
2168                 The Wood
7130               Bride Wars
3425    The Magnificent Seven
7102             Seven Pounds
7715      No Strings Attached
159                  Mallrats
3696         All Over the Guy
Name: title, dtype: object

In [16]:
get_recommendations('The Godfather')

973      The Godfather: Part II
8387                 The Family
3509                       Made
4196         Johnny Dangerously
5667                       Fury
29               Shanghai Triad
1582    The Godfather: Part III
4221                    8 Women
2159              Summer of Sam
618                     Thinner
Name: title, dtype: object

# **Credits and Keywords Based Recommender**

In [17]:
credits = pd.read_csv('/kaggle/input/the-movies-dataset/credits.csv')
keywords = pd.read_csv('/kaggle/input/the-movies-dataset/keywords.csv')
keywords['id'] = keywords['id'].astype('int')
credits['id'] = credits['id'].astype('int')
df['id'] = df['id'].astype('int')

df.shape

(45463, 24)

In [18]:
df = df.merge(credits, on='id')
df = df.merge(keywords, on='id')
sdf = df[df['id'].isin(df_small)]
sdf.shape

(9219, 27)

In [19]:
features = ['cast', 'crew', 'keywords']
for feature in features:
    sdf[feature] = sdf[feature].apply(literal_eval)

sdf['cast_size'] = sdf['cast'].apply(lambda x: len(x))
sdf['crew_size'] = sdf['crew'].apply(lambda x: len(x))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value inst

In [20]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        if len(names) > 3:
            names = names[:3]
        return names
    return []

sdf['director'] = sdf['crew'].apply(get_director)

features = ['cast', 'keywords', 'genres']
for feature in features:
    sdf[feature] = sdf[feature].apply(get_list)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [21]:
sdf[['title', 'cast', 'director', 'keywords', 'genres']].head(3)

,title,cast,director,keywords,genres
0,Toy Story,"[Tom Hanks, Tim Allen, Don Rickles]",John Lasseter,"[jealousy, toy, boy]",[]
1,Jumanji,"[Robin Williams, Jonathan Hyde, Kirsten Dunst]",Joe Johnston,"[board game, disappearance, based on children'...",[]
2,Grumpier Old Men,"[Walter Matthau, Jack Lemmon, Ann-Margret]",Howard Deutch,"[fishing, best friend, duringcreditsstinger]",[]


In [22]:
s = sdf.apply(lambda x: pd.Series(x['keywords']),axis=1).stack().reset_index(level=1, drop=True)
s.name = 'keyword'
s = s.value_counts()
s[:10]

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.


independent film    287
based on novel      271
sex                 197
woman director      175
musical             170
prison              155
new york            149
female nudity       145
gay                 132
sport               127
Name: keyword, dtype: int64

In [23]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else: 
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
features = ['cast', 'keywords', 'director', 'genres']
for feature in features:
    sdf[feature] = sdf[feature].apply(clean_data)
    
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
sdf['soup'] = sdf.apply(create_soup, axis=1)
count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(sdf['soup'])

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)

sdf = sdf.reset_index()
indices = pd.Series(sdf.index, index=sdf['title'])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if sys.path[0] == '':
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()


In [24]:
get_recommendations('The Prestige', cosine_sim2)

6218                            Batman Begins
6981                          The Dark Knight
8031                    The Dark Knight Rises
2290                           Stir of Echoes
3577                  Dirty Rotten Scoundrels
5887                       Prince of Darkness
1861    Hands on a Hard Body: The Documentary
4633                               Spellbound
5073                            The Statement
5405                           Educating Rita
Name: title, dtype: object

# **Collabrative Filtering**

In [25]:
reader = Reader()
ratings = pd.read_csv('/kaggle/input/the-movies-dataset/ratings_small.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [26]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
print(data.df.head())
#.split(n_folds=5)

   userId  movieId  rating
0       1       31     2.5
1       1     1029     3.0
2       1     1061     3.0
3       1     1129     2.0
4       1     1172     4.0


In [27]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'])

{'test_rmse': array([0.89504957, 0.89370731, 0.8941894 , 0.89790029, 0.90044928]),
 'test_mae': array([0.68775724, 0.68464773, 0.68817964, 0.69258948, 0.69468056]),
 'fit_time': (10.064850091934204,
  9.797006845474243,
  10.114240646362305,
  9.730942487716675,
  9.875789880752563),
 'test_time': (0.30887722969055176,
  0.3053605556488037,
  0.3043251037597656,
  0.2924196720123291,
  0.29932546615600586)}

In [28]:
trainset = data.build_full_trainset()
svd.fit(trainset)

ratings[ratings['userId'] == 1]

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205
5,1,1263,2.0,1260759151
6,1,1287,2.0,1260759187
7,1,1293,2.0,1260759148
8,1,1339,3.5,1260759125
9,1,1343,2.0,1260759131


In [29]:
svd.predict(1, 302, 3)

Prediction(uid=1, iid=302, r_ui=3, est=2.748863965559986, details={'was_impossible': False})

# **Hybrid Model**

In [33]:
def convert_int(x):
    try:
        return int(x)
    except:
        return np.nan
    
id_map = pd.read_csv('/kaggle/input/the-movies-dataset/links_small.csv')[['movieId', 'tmdbId']]
id_map['tmdbId'] = id_map['tmdbId'].apply(convert_int)
id_map.columns = ['movieId', 'id']
id_map = id_map.merge(sdf[['title', 'id']], on='id').set_index('title')

indices_map = id_map.set_index('id')

def hpred(userId, title):
    index = indices[title]
    Id = id_map.loc[title]['id']
    movie_id = id_map.loc[title]['movieId']
    
    recc = list(enumerate(cos_sim[int(index)]))
    recc = sorted(recc, key=lambda x: x[1], reverse=True)
    recc = recc[1:25]
    movie_indices = [i[0] for i in recc]
    movies = sdf.iloc[movie_indices][['title', 'vote_count', 'vote_average', 'id']]
    movies['est'] = movies['id'].apply(lambda x: svd.predict(userId, indices_map.loc[x]['movieId']).est)
    movies = movies.sort_values('est', ascending=False)
    return movies.head(10)

In [34]:
hpred(1, 'Bride Wars')

,title,vote_count,vote_average,id,est
10,The American President,199.0,6.5,9087,3.039180
16,Sense and Sensibility,364.0,7.2,4584,2.995136
15,Casino,1343.0,7.8,524,2.958801
12,Balto,423.0,7.1,21032,2.930562
5,Heat,1886.0,7.7,949,2.876470
20,Get Shorty,305.0,6.4,8012,2.744558
24,Leaving Las Vegas,365.0,7.1,451,2.725151
21,Copycat,199.0,6.5,1710,2.692969
7,Tom and Huck,45.0,5.4,45325,2.616735
4,Father of the Bride Part II,173.0,5.7,11862,2.592445


In [35]:
hpred(500, 'Bride Wars')

,title,vote_count,vote_average,id,est
5,Heat,1886.0,7.7,949,3.472655
15,Casino,1343.0,7.8,524,3.412607
24,Leaving Las Vegas,365.0,7.1,451,3.245808
16,Sense and Sensibility,364.0,7.2,4584,3.143665
12,Balto,423.0,7.1,21032,3.121053
10,The American President,199.0,6.5,9087,3.096111
21,Copycat,199.0,6.5,1710,2.946240
19,Money Train,224.0,5.4,11517,2.896158
2,Grumpier Old Men,92.0,6.5,15602,2.881259
23,Powder,143.0,6.3,12665,2.776734


Here, in hybrid recommender system, we get different recommendations for different users although the movie is the same.

# **CONCLUSION**

An information filtering system used to predict the preference of movies that a user would watch was hence modelled.

# **Reference/ Acknowledgements**

1.https://hackernoon.com/introduction-to-recommender-system-part-1-collaborative-filtering-singular-value-decomposition-44c9659c5e75

2.https://www.kaggle.com/ibtesama/getting-started-with-a-movie-recommendation-system

